In [1]:
#IMPORTAMOS LAS LIBRERIAS BASADAS EN TENSORFLOW Y KERAS
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

In [2]:
#LIMPIAMOS SESION DE KERAS PARA EMPEZAR DESDE 0 EN MEMORIA
K.clear_session()
#GENERAMOS 2 VARIABLES CON CADA UNA DE LAS CARPETAS TANTO DE ENTRENAMIENTO COMO DE TEST CON FOTOGRAFIAS
data_entrenamiento = './CarneDataset/train'
data_validacion = './CarneDataset/test'
#DECLARACION DE VARIABLES CON VALORES QUE PARAMETRIZARAN EL DESEMPEÑO DEL ENTRENAMIENTO DEL MODELO DE CLASIFICACION DE FOTOS
epocas=15 #NUMERO DE EPOCHS PARA EL TRAINING
longitud, altura = 100, 100 #REDIMENSIONAMIENTO DE FOTOGRAFIAS
batch_size = 10 # TAMAÑO DE FOTOGRAFIAS A GESTIONAR
pasos = 120 #NUMERO DE STEPS PARA ENTRENAMIENTO
validation_steps = 80 #NUMERO DE STEPS PARA VALIDACION
clases = 8 #CANTIDAD DE CLASES DE FOTOGRAFIAS A CLASIFICAR
lr = 0.001 #LERANING RATE PARA EL PROCESO DE ENTRENAMIENTO
# Declaracion de variables con valores para el modelo
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)

In [3]:
#FUNCION IMAGEDATAGENERATOR QUE PROPORCIONARA MODIFICACION A FOTOGRAFIAS PARA OPTIMIZAR EL PROCESO DE ENTRENAMIENTO
entrenamiento_datagen = ImageDataGenerator(
    rotation_range=20,  # Rango de rotación aleatoria en grados
    width_shift_range=0.1,  # Rango de desplazamiento horizontal aleatorio
    height_shift_range=0.1,  # Rango de desplazamiento vertical aleatorio
    rescale=1. / 255, # Rango de escala de la imagen
    shear_range=0.2, # Rango de efecto de inclinación o distorsión
    zoom_range=0.2, # Rango de efecto de zoom
    horizontal_flip=True) # aplicar volteo horizontal a las imágenes

test_datagen = ImageDataGenerator(rescale=1. / 255)
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA ENTRENAMIENTO
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')
#FUNCION PARA CONFIGURACION DEL PROCESO DE RECUPERACION DE INFORMACION DE FOTOS PARA TEST
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 1634 images belonging to 8 classes.
Found 810 images belonging to 8 classes.


In [4]:
#FUNCION PARA EMPEZAR EL MODELO Y SU CONSTRUCCION
cnn = Sequential()
#PRIMERA CAPA
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#SEGUNDA CAPA
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same", activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
#RECONFIGURACION DE LA INFORMACION EN DATOS RECONOCIBLES POR EL MODELO
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))
#COMPILACION DE LA INFORMACION DENTRO DEL MODELO PARA SU ENTRENAMIENTO 
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])

In [5]:
from keras.callbacks import LearningRateScheduler
# Definir una función para reducir el learning rate gradualmente
def lr_scheduler(epoch):
    if epoch < 10:
        return lr
    else:
        return lr * 0.1

# Crear un objeto LearningRateScheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

In [6]:
#ENTRENAMIENTO DEL MODELO SEGUN LOS DATOS INGRESADOS COMO PARAMETROS
historia = cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps,
    callbacks=[lr_scheduler_callback]
)

Epoch 1/15
120/120 [==============================] - 46s 368ms/step - loss: 1.4860 - accuracy: 0.5595 - val_loss: 1.1850 - val_accuracy: 0.6463 - lr: 0.0010
Epoch 2/15
120/120 [==============================] - 44s 364ms/step - loss: 1.0081 - accuracy: 0.6450 - val_loss: 0.7328 - val_accuracy: 0.7225 - lr: 0.0010
Epoch 3/15
120/120 [==============================] - 44s 368ms/step - loss: 0.8638 - accuracy: 0.6817 - val_loss: 0.7363 - val_accuracy: 0.7538 - lr: 0.0010
Epoch 4/15
120/120 [==============================] - 44s 366ms/step - loss: 0.7167 - accuracy: 0.7286 - val_loss: 1.1120 - val_accuracy: 0.5975 - lr: 0.0010
Epoch 5/15
120/120 [==============================] - 44s 364ms/step - loss: 0.6295 - accuracy: 0.7672 - val_loss: 1.0167 - val_accuracy: 0.6087 - lr: 0.0010
Epoch 6/15
120/120 [==============================] - 45s 375ms/step - loss: 0.6485 - accuracy: 0.7533 - val_loss: 0.8630 - val_accuracy: 0.6550 - lr: 0.0010
Epoch 7/15
120/120 [==============================] 

In [7]:
# Mostrar el accuracy y la loss
acc = historia.history['accuracy']
loss = historia.history['loss']
val_acc = historia.history['val_accuracy']
val_loss = historia.history['val_loss']

# Imprimir los resultados
print(f"Accuracy: {acc[-1]:.2f}")
print(f"Loss: {loss[-1]:.2f}")
print(f"Validation Accuracy: {val_acc[-1]:.2f}")
print(f"Validation Loss: {val_loss[-1]:.2f}")

Accuracy: 0.89
Loss: 0.30
Validation Accuracy: 0.73
Validation Loss: 0.77


In [8]:
#CREACION Y ALMACENAMIENTO DEL RESULTANTE DEL ENTRENAMIENTO PARA RECONOCIMIENTO DE FOTOGRAFIAS
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')